In [3]:
from blockulib.playing import DeepSearch
from blockulib.data import DataTransformer
import blockulib.models as blom
import blockulib
import torch

In [4]:
"""class DeepSearch(ModelBasedLoop):
    def __init__(self):
        super().__init__()
        self.probe = Probe()
        self.dt = DataTransformer()
    
    def rethink_logits(self, data, old_logits, rethink_batch, probes_per_pos = 5):
        if (data.shape[0] == 0):
            return torch.tensor([])
        gls, lls = [], []
        for i in range(0, data.shape[0], rethink_batch):
            batch = data[i:i+rethink_batch]
            batch = batch.repeat_interleave(probes_per_pos, dim = 0)
            gl, ll = self.probe(pos_tensor = batch, depth = 3, temperature = 0.7, top_k = 3)
            gls.append(gl)
            lls.append(ll)
        GLs = torch.cat(gls)
        LLs = torch.cat(lls)
        GLs += self.dt.logits_to_nums(LLs)
        LLs = self.dt.nums_to_logits(GLs)
        LLs = LLs.view(data.shape[0], probes_per_pos).mean(dim = 1)
        return LLs """

'class DeepSearch(ModelBasedLoop):\n    def __init__(self):\n        super().__init__()\n        self.probe = Probe()\n        self.dt = DataTransformer()\n    \n    def rethink_logits(self, data, old_logits, rethink_batch, probes_per_pos = 5):\n        if (data.shape[0] == 0):\n            return torch.tensor([])\n        gls, lls = [], []\n        for i in range(0, data.shape[0], rethink_batch):\n            batch = data[i:i+rethink_batch]\n            batch = batch.repeat_interleave(probes_per_pos, dim = 0)\n            gl, ll = self.probe(pos_tensor = batch, depth = 3, temperature = 0.7, top_k = 3)\n            gls.append(gl)\n            lls.append(ll)\n        GLs = torch.cat(gls)\n        LLs = torch.cat(lls)\n        GLs += self.dt.logits_to_nums(LLs)\n        LLs = self.dt.nums_to_logits(GLs)\n        LLs = LLs.view(data.shape[0], probes_per_pos).mean(dim = 1)\n        return LLs '

In [5]:
DS = DeepSearch()

In [6]:
import time


for i in range(1, 31):
    num_games = 10*i
    print(f"Starting epoch {i} with {num_games} games")
    start_time = time.time()
    
    data = DS(num_games = num_games, rethink_batch = 400, temperature = 0.7, top_k = 3)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.6f} seconds - {(elapsed_time/i):.6f} per 10 games")

Starting epoch 1 with 10 games
Elapsed time: 1.553818 seconds - 1.553818 per 10 games
Starting epoch 2 with 20 games
Elapsed time: 2.223736 seconds - 1.111868 per 10 games
Starting epoch 3 with 30 games
Elapsed time: 2.993550 seconds - 0.997850 per 10 games
Starting epoch 4 with 40 games
Elapsed time: 4.357768 seconds - 1.089442 per 10 games
Starting epoch 5 with 50 games
Elapsed time: 6.473171 seconds - 1.294634 per 10 games
Starting epoch 6 with 60 games
Elapsed time: 5.841307 seconds - 0.973551 per 10 games
Starting epoch 7 with 70 games
Elapsed time: 7.297310 seconds - 1.042473 per 10 games
Starting epoch 8 with 80 games
Elapsed time: 7.320915 seconds - 0.915114 per 10 games
Starting epoch 9 with 90 games
Elapsed time: 8.255341 seconds - 0.917260 per 10 games
Starting epoch 10 with 100 games
Elapsed time: 9.810492 seconds - 0.981049 per 10 games
Starting epoch 11 with 110 games
Elapsed time: 11.049978 seconds - 1.004543 per 10 games
Starting epoch 12 with 120 games
Elapsed time: 12

KeyboardInterrupt: 